# Import Libraries


In [1]:
import pandas as pd # data processing
import numpy as np # Mathematical operations
import seaborn as sns # Data visualization
import matplotlib.pyplot as plt # Data visualization
import moviepy.editor as mpy # Video and audio editing
import requests # HTTP requests
import os # File operations
import sys # System operations
import boto3 # AWS SDK
from pydub import AudioSegment # Audio processing

# Media Files

    A. Issues when loading the video: 

                1. When previewing the video it has a mixer issue - Will skip  this for now till fix is found, will convert the file and store to bucket
                2. Where do we store the temporary files - Files will be stored in a bucket on S3 on AWS

**File convertion**

In [ ]:
file_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/"
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"

#Loop through each file within the directory
for file in os.listdir(file_path):
    if file.endswith(".mp4"):
        video_path = os.path.join(file_path, file)

        #Load video file
        try:
            video = mpy.VideoFileClip(video_path)
            print(f'Video Loaded: {video_path}')
            audio = video.audio
            audio.write_audiofile(os.path.join(save_path,f'{file}.mp3'))
            
            # #Access AWS S3 Bucket
            # aws_access_key_id = ''
            # aws_secret_access_key = ''

            # s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

            # with open(os.path.join(file_path,f'{file}.mp3'), 'rb') as data:
            #     s3.Bucket('dancevibedaily').put_object(Key=f'{file}.mp3', Body=data)
            #     print(f'Audio file uploaded to S3: {file}.mp3')

            # #Remove audio file from local storage
            # os.remove(os.path.join(file_path,f'{file}.mp3'))
            # print("Audio file removed from local storage")

        except Exception as e:
            print(f"Error loading video: {e}")
            continue

# Data Building and Cleaning

**Issues on API use with bucket**

                1. As I am new to Data Engineering I have not figured out how to work with api for now will work with local directory.
                2. There is a 'NoneType' object is not subscriptable, will go through these json and get to understand the issue.

In [27]:
import requests # HTTPS Requests
data = {
    'api_token': '',
    'return': 'musicbrainz',
}

# Will loop through the files in the directory and extract the song title, artist, and album from the API response
save_path = "C:/Users/tedjo/Downloads/dancevibedaily/media/audio/"
results = []

for i in os.listdir(save_path):
    if i.endswith(".mp3"):
        files = {
            'file': open(os.path.join(save_path,i),'rb') #reading binary file
        }
        try:
            result = requests.post('https://api.audd.io/', data=data, files=files) # Send data to the API 
            if result.json()['status'] == 'error':
                print(f"Error processing file: {result.json()}")
                continue
            print(result.json()['result']['title'])
            results.append({
                'Title': result.json()['result']['title'],
                'Artist': result.json()['result']['artist'],
                'Album': result.json()['result']['album'],
                'Release Date': result.json()['result']['release_date'],
            })# Append the results to the list, which will be used to create a DataFrame
        except Exception as e:
            print(f"Error processing file: {e}")
            continue

df = pd.DataFrame(results) # Create a DataFrame from the results list


All Over You
ULAZI Infinity MusiQ (yey)
Tjina
YESHUA (dance cruise)
Private school
Error processing file: 'NoneType' object is not subscriptable
Look at Her
Killin Dem
Error processing file: 'NoneType' object is not subscriptable
Escapism. (Sped Up)
Error processing file: 'NoneType' object is not subscriptable
Error processing file: 'NoneType' object is not subscriptable
Jinja
Dalie (feat. Baby S.O.N)
soso
الي اين انا ذاهب
Water
Issa Necessary (feat. Kiss Daniel)
Mnike (UK Radio Edit)
Barbie World (with Aqua) [From Barbie The Album]
UNAVAILABLE (feat. Musa Keys)
Look at Her
Hwiralang feat. Siko Wa Mmino,Hlogi Mash
dabala
Mnike
Cough
Error processing file: 'NoneType' object is not subscriptable
Water
Venda Boy (feat. fefe laku dura)
AH TXE TXE (feat. TYSON)
Floor Seats
Error processing file: 'NoneType' object is not subscriptable
Rock Attack
Walking dead
Kafe'
Shu! (feat. Chley)
Error processing file: 'NoneType' object is not subscriptable
Mnike
Get Up (Main Version)
Lost In The Bass. M

In [29]:
df.to_csv('C:/Users/tedjo/Downloads/dancevibedaily/media/audio/audio_metadata.csv', index=False) # Save the DataFrame to a CSV file

# Data Cleaning